In [1]:
!ls

batch_log.json	FindAnomoly.ipynb  path.png  stream_log.json


In [2]:
import pandas as pd

In [32]:
!head -n 10 stream_log.json

{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "5008", "amount": "47.99"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "7891", "amount": "65.22"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "1677", "amount": "70.52"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "4485", "amount": "40.75"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "5846", "amount": "18.55"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "8840", "amount": "89.90"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "6764", "amount": "29.74"}
{"event_type":"unfriend", "timestamp":"2017-06-14 18:46:50", "id1": "5256", "id2": "3150"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "2743", "amount": "117.64"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "1111", "amount": "13.15"}


# Processing the Batch file

In [36]:
import json
import pandas as pd
import numpy as np

fileName='batch_log.json'
Param=[]
Purchase = []
NetworkBe=[]
NetworkNot=[]

with open(fileName) as f:
    for line in f:
            
        if line.startswith('{"D":'):
            LineElem=json.loads(line)
            D,T=int(LineElem['D']),int(LineElem['T'])
            Param=[D,T]
        elif line.startswith('{"event_type":"purchase"'):
            LineElem=json.loads(line)
            Purchase.append(np.array([LineElem["timestamp"],LineElem["id"],LineElem["amount"]]))
        elif line.startswith('{"event_type":"befriend"'):
            LineElem=json.loads(line)
            NetworkBe.append(np.array([LineElem["timestamp"],LineElem["id1"],LineElem["id2"]]))
        elif line.startswith('{"event_type":"unfriend"'):
            LineElem=json.loads(line)
            NetworkNot.append(np.array([LineElem["timestamp"],LineElem["id1"],LineElem["id2"]]))


In [40]:
DF_Purch=pd.DataFrame(Purchase,columns=['timestamp','id','amount'])
DF_NetworkBe=pd.DataFrame(NetworkBe,columns=['timestamp','id1','id2'])
NetworkNot=pd.DataFrame(NetworkNot,columns=['timestamp','id1','id2'])


# Building scocial Network Graph

In [64]:
set1= set(DF_NetworkBe.id1.tolist())
set2= set(DF_NetworkBe.id2.tolist())
SetAll=set1.union(set2)
ListAll=list(SetAll)

DF_Id_list=DF_NetworkBe.groupby('id1')['id2'].apply(list).reset_index()

import networkx as nx
G=nx.Graph()
# add a list of nodes,
G.add_nodes_from(ListAll)

EdgesAdded=[(DF_NetworkBe['id1'][i],DF_NetworkBe['id2'][i]) for i in range(len(DF_NetworkBe))]

G.add_edges_from(EdgesAdded)

EdgesrEMOVED=[(NetworkNot['id1'][i],NetworkNot['id2'][i]) for i in range(len(NetworkNot))]

G.remove_edges_from(EdgesrEMOVED)

# Processing the real time stream file 

In [74]:
fileNameStream='stream_log.json'
Purchase2 = []
NetworkBe2=[]
NetworkNot2=[]

def addEdge(G,Edge):
    G.add_edges_from(Edge)

def rmEdge(G,Edge):
    G.remove_edges_from(Edge)
                     
def addNode(G,Node):
    G.add_node(Node)

def addNodes(G,Nodes):
    G.add_nodes_from(ListAll)
                    

# Param=[D,T]
# T and D are gllobal vraibles

def CheckAnomoly(timestamp,node_id,amount,T, D):
    # Build dataFrame with all id in the social network with D distance from node_id
    
    
    # Filter the graph with T nearest transactions to the time stamp 
    
    # calculate the mean and the Std of the network 
    
    # if the amount is > mean + 3*std  falg the transaction as Anomoly
    
    
    
        


with open(fileNameStream) as f:
    for line in f:
  
        if line.startswith('{"event_type":"purchase"'):
            LineElem=json.loads(line)
            Purchase2.append(np.array([LineElem["timestamp"],LineElem["id"],LineElem["amount"]]))
            addNode(G,LineElem["id"])
            CheckAnomoly(LineElem["timestamp"],LineElem["id"],LineElem["amount"])
           
        elif line.startswith('{"event_type":"befriend"'):
            LineElem=json.loads(line)
            NetworkBe2.append(np.array([LineElem["timestamp"],LineElem["id1"],LineElem["id2"]]))
            # Updating the scial network graph (adding)
            addNodes(G,[LineElem["id1"],LineElem["id2"]])
            addEdge(G,[(LineElem["id1"],LineElem["id2"])])

           
            
        elif line.startswith('{"event_type":"unfriend"'):
            LineElem=json.loads(line)
            NetworkNot2.append(np.array([LineElem["timestamp"],LineElem["id1"],LineElem["id2"]]))
            # Updating the scial network graph (removing) 
            addNodes(G,[LineElem["id1"],LineElem["id2"]])
            rmEdge(G,[(LineElem["id1"],LineElem["id2"])])
        
    
        

In [73]:
 addEdge(G,[(LineElem["id1"],LineElem["id2"])])

In [58]:
(NetworkBe2[i2][1],NetworkBe2[i2][2])


('3556', '2143')

In [43]:
'''
G.number_of_nodes()
G.number_of_edges()

DF_Id_list.id2[DF_Id_list.id1=='1'].tolist()

DF_Id_list["Length"]=DF_Id_list.id2.apply(len)

len(DF_Id_list.id2[0])

G.neighbors('1')

nx.single_source_shortest_path_length(G, '1', cutoff=1)

nx.single_source_shortest_path_length(G, '1', cutoff=2)

%matplotlib inline
import matplotlib.pyplot as plt
nx.draw(G)
plt.show()
plt.savefig("path.png")
'''

'\nG.number_of_nodes()\nG.number_of_edges()\n\nDF_Id_list.id2[DF_Id_list.id1==\'1\'].tolist()\n\nDF_Id_list["Length"]=DF_Id_list.id2.apply(len)\n\nlen(DF_Id_list.id2[0])\n\nG.neighbors(\'1\')\n\n%matplotlib inline\nimport matplotlib.pyplot as plt\nnx.draw(G)\nplt.show()\nplt.savefig("path.png")\n'

In [31]:
nx.single_source_shortest_path_length(G, '1', cutoff=2)

{'1': 0,
 '1000': 2,
 '1038': 2,
 '104': 2,
 '1041': 2,
 '1048': 2,
 '1057': 2,
 '1070': 2,
 '1073': 2,
 '1075': 2,
 '1139': 2,
 '1169': 2,
 '1174': 2,
 '1201': 2,
 '1208': 2,
 '1355': 2,
 '1452': 2,
 '1491': 2,
 '1504': 2,
 '1578': 2,
 '1592': 2,
 '1603': 2,
 '1605': 2,
 '1689': 2,
 '1699': 2,
 '1710': 2,
 '1713': 2,
 '1844': 2,
 '187': 2,
 '1919': 2,
 '1948': 2,
 '1957': 1,
 '1958': 2,
 '1970': 2,
 '2020': 2,
 '2054': 2,
 '2109': 2,
 '2115': 2,
 '2139': 2,
 '2160': 2,
 '2178': 2,
 '2230': 2,
 '2245': 2,
 '2257': 2,
 '2267': 2,
 '23': 2,
 '2438': 2,
 '2542': 2,
 '2550': 1,
 '2570': 2,
 '269': 2,
 '2695': 2,
 '2720': 2,
 '2739': 2,
 '2745': 2,
 '2752': 2,
 '2765': 2,
 '2770': 1,
 '2809': 2,
 '2824': 2,
 '2853': 2,
 '2862': 2,
 '2875': 2,
 '2878': 2,
 '2894': 2,
 '2983': 2,
 '2997': 2,
 '303': 2,
 '3064': 2,
 '3102': 2,
 '3142': 2,
 '315': 2,
 '3153': 2,
 '3190': 2,
 '3267': 2,
 '3299': 2,
 '3340': 2,
 '3342': 2,
 '3390': 2,
 '3402': 2,
 '3477': 2,
 '3482': 2,
 '3484': 2,
 '355': 2,
 '3